<a href="https://colab.research.google.com/github/AyanBhunia/Super-brain-ML/blob/master/Pizza_vs_Softdrink.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !unzip "/content/drive/My Drive/super-brain/food_classifer_dataset.zip" -d "/content/"

# Importing Important Libraries

In [ ]:
# this is going to help in data processing
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
#these are going to help in the model building.
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense


# Setting up Variables

In [ ]:
# dimensions of our images.
img_width, img_height = 150, 150

#setting up the directories
train_data_dir = '/content/classifer_dataset/train'
validation_data_dir = '/content/classifer_dataset/validate'

#setting up the batchsizes.
nb_train_samples = 8400
nb_validation_samples = 1600

#Hyperparameter
epochs = 50
batch_size = 16

In [ ]:
input_shape = (img_width, img_height, 3)

In [ ]:
print(input_shape)

(150, 150, 3)


# Data Augmentation

In [ ]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1./ 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [ ]:
#this generates batches of augment data for training
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 8400 images belonging to 2 classes.


In [ ]:
# this is the augmentation configuration we will use for validating
val_datagen = ImageDataGenerator(rescale=1./255)

#this generates batches of augment data for validating
validation_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')


Found 1600 images belonging to 2 classes.


# Building the CNN model

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

In [ ]:
model.add(Flatten())

In [ ]:
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
#configuring the model
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


In [ ]:
#to print a summary representation of your model
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
batch_normalization_8 (Batch (None, 148, 148, 32)      128       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
batch_normalization_9 (Batch (None, 72, 72, 64)        256       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 36, 36, 64)       

# Training the model (Forward + Back Propagation)

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
525/525 [==============================] - 101s 192ms/step - loss: 0.4373 - accuracy: 0.8392 - val_loss: 1.0111 - val_accuracy: 0.6744
Epoch 2/50
525/525 [==============================] - 100s 191ms/step - loss: 0.2562 - accuracy: 0.8992 - val_loss: 0.2350 - val_accuracy: 0.9013
Epoch 3/50
525/525 [==============================] - 100s 191ms/step - loss: 0.2218 - accuracy: 0.9112 - val_loss: 1.1704 - val_accuracy: 0.6775
Epoch 4/50
525/525 [==============================] - 101s 192ms/step - loss: 0.2065 - accuracy: 0.9167 - val_loss: 0.2189 - val_accuracy: 0.9200
Epoch 5/50
525/525 [==============================] - 100s 191ms/step - loss: 0.1982 - accuracy: 0.9181 - val_loss: 0.1731 - val_accuracy: 0.9356
Epoch 6/50
525/525 [==============================] - 100s 190ms/step - loss: 0.1850 - accuracy: 0.9292 - val_loss: 0.1870 - val_accuracy: 0.9262
Epoch 7/50
525/525 [============================

# Saving Model

In [ ]:
#to save the weights in the model as a HDFS file

model.save_weights('model_weight.h5')

In [ ]:
#to save the architecture of the model as a json file

with open('model_architecture.json','w') as f:
    f.write(model.to_json())

# Restoring the Model

In [ ]:
from keras.models import load_model
from keras.models import model_from_json

In [ ]:
# Model reconstruction from JSON file
with open('/content/model_architecture.json', 'r') as f:
    model = model_from_json(f.read())

# Load weights into the new model
model.load_weights('/content/model_weight.h5')

# Testing the model

In [ ]:
import numpy as np
from keras.preprocessing import image
a="/content/fresh-pizza-cold-cola-drink-18496799.jpg"
test_image=image.load_img(a,target_size=(img_width,img_height))
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
result=model.predict(test_image)
print(result)

if result[0][0]==1.0:
    prediction='SoftDrink'
else:
    prediction='Pizza'
print("You got a "+ prediction + " Yupeeeeeeeeeeeeeeeeeeeeeeeeeeeeee!!!")

[[1.]]
You got a SoftDrink Yupeeeeeeeeeeeeeeeeeeeeeeeeeeeeee!!!
